<a href="https://colab.research.google.com/github/rnarkk/jupyter/blob/main/Bayesian-Methods-for-Hackers/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probabilistic Programming and Bayesian Methods for Hackers Chapter 4

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter4_TheGreatestTheoremNeverTold/Ch4_LawOfLargeNumbers_TFP.ipynb"><img height="32px" src="https://colab.research.google.com/img/colab_favicon.ico" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter4_TheGreatestTheoremNeverTold/Ch4_LawOfLargeNumbers_TFP.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br>
<br>
<br>

Original content ([this Jupyter notebook](https://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter4_TheGreatestTheoremNeverTold/Ch4_LawOfLargeNumbers_PyMC2.ipynb)) created by Cam Davidson-Pilon ([`@Cmrn_DP`](https://twitter.com/Cmrn_DP))

Welcome to Bayesian Methods for Hackers. The full Github repository is available at [github/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers). The other chapters can be found on the project's [homepage](https://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/). We hope you enjoy the book, and we encourage any contributions!

---------
### Table of Contents
- The greatest theorem never told
  - The Law of Large Numbers
  - Intuition
  - How do we compute $Var(Z)$ though?
  - Expected values and probabilities
  - What does this all have to do with Bayesian statistics?
  - The Disorder of Small Numbers
  - Example: Aggregated geographic data
  - Example: Kaggle's U.S. Census Return Rate Challenge
  - Example: How to order Reddit submissions
    - Setting up the Praw Reddit API
    - Register your Application on Reddit
      - Reddit API Setup
    - Sorting!
    - But this is too slow for real-time!
  - Extension to Starred rating systems
  - Example: Counting Github stars
  - Conclusion
  - Appendix
    - Exercises
    - Kicker Careers Ranked by Make Percentage
    - Average Household Income by Programming Language
  - References

______



In [ ]:
!pip3 install -U pip
!pip3 uninstall -y tensorflow tensorflow-probability
!pip3 install -U jax tfp-nightly[jax]

!pip3 install -q praw wget

     |████████████████████████████████| 1.6MB 14.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Found existing installation: tensorflow-probability 0.12.1
Uninstalling tensorflow-probability-0.12.1:
  Successfully uninstalled tensorflow-probability-0.12.1
     |████████████████████████████████| 680 kB 13.7 MB/s 
     |████████████████████████████████| 5.5 MB 22.8 MB/s 
  Created wheel for jax: filename=jax-0.2.16-py3-none-any.whl size=784392 sha256=ab0f6793d52b43ce2e8dc2fe7777ac143e77729120725ae6b9d8d11397e6d199
  Stored in directory: /root/.cache/pip/wheels/fd/23/9e/4da95cc1faef02199df99d4928b8e870a7c3de039403dcbc6c
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.2.13
    Uninstalling jax-0.2.13:
      Successfully uninstalled jax-0.2.13


In [ ]:
"""
The book uses a custom matplotlibrc file, which provides the unique styles for
matplotlib plots. If executing this book, and you wish to use the book's
styling, provided are two options:
    1. Overwrite your own matplotlibrc file with the rc-file provided in the
       book's styles/ dir. See http://matplotlib.org/users/customizing.html
    2. Also in the styles is  bmh_matplotlibrc.json file. This can be used to
       update the styles in only this notebook. Try running the following code:

        import json
        s = json.load(open("../styles/bmh_matplotlibrc.json"))
        matplotlib.rcParams.update(s)
"""
import datetime
import json

import jax.numpy as np
from jax.numpy import (
    int32 as i32,
    float32 as f32)
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
Binomial, Exponential, Normal, Poisson, Uniform = \
    tfd.Binomial, tfd.Exponential, tfd.Normal, tfd.Poisson, tfd.Uniform
tfb = tfp.bijectors

from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import seaborn as sns

import wget

plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'

%matplotlib inline

sns.set_context('notebook')

TFColor = [
    '#F15854',
    '#5DA5DA',
    '#FAA43A',
    '#60BD68',
    '#F17CB0',
    '#B2912F',
    '#B276B2',
    '#DECF3F',
    '#4D4D4D']

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorflow-probability-0.12.1:
  Successfully uninstalled tensorflow-probability-0.12.1
     |████████████████████████████████| 458.7MB 34kB/s 
     |████████████████████████████████| 5.5MB 21.5MB/s 
     |████████████████████████████████| 4.2MB 19.5MB/s 
     |████████████████████████████████| 5.5MB 27.5MB/s 
     |████████████████████████████████| 471kB 45.1MB/s 
     |████████████████████████████████| 1.3MB 27.6MB/s 
     |████████████████████████████████| 13.4MB 240kB/s 
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: keras-nightly 2.5.0.dev2021032900
    Uninstalling keras-nightly-2.5.0.dev2021032900:
      Successfully uninstalled keras-nightly-2.5.0.dev2021032900
     |████████████████████████████████| 174kB 4.7MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 


ImportError: ignored

## The greatest theorem never told


This chapter focuses on an idea that is always bouncing around our minds, but is rarely made explicit outside books devoted to statistics. In fact, we've been using this simple idea in every example thus far. 

### The Law of Large Numbers

Let $Z_i$ be $N$ independent samples from some probability distribution. According to *the Law of Large numbers*,  so long as the expected value $E[Z]$ is finite, the following holds,

$$\frac{1}{N} \sum_{i=1}^N Z_i \rightarrow E[ Z ],  \;\;\; N \rightarrow \infty.$$

In words:

>   The average of a sequence of random variables from the same distribution converges to the expected value of that distribution.

This may seem like a boring result, but it will be the most useful tool you use.

### Intuition 

If the above Law is somewhat surprising,  it can be made more clear by examining a simple example. 

Consider a random variable $Z$ that can take only two values, $c_1$ and $c_2$. Suppose we have a large number of samples of $Z$, denoting a specific sample $Z_i$. The Law says that we can approximate the expected value of $Z$ by averaging over all samples. Consider the average:


$$ \frac{1}{N} \sum_{i=1}^N Z_i $$


By construction, $Z_i$ can only take on $c_1$ or $c_2$, hence we can partition the sum over these two values:
$$
\begin{align}
\frac{1}{N} \sum_{i=1}^N Z_i & =\frac{1}{N} \big(  \sum_{ Z_i = c_1}c_1 + \sum_{Z_i=c_2}c_2 \big) \\
& = c_1 \sum_{ Z_i = c_1}\frac{1}{N} + c_2 \sum_{ Z_i = c_2}\frac{1}{N} \\
& = c_1 \times \text{ (approximate frequency of $c_1$) } \\
& \;\;\;\;\;\;\;\;\; + c_2 \times \text{ (approximate frequency of $c_2$) } \\
& \approx c_1 \times P(Z = c_1) + c_2 \times P(Z = c_2 ) \\
& = E[Z]
\end{align}
$$

Equality holds in the limit, but we can get closer and closer by using more and more samples in the average. This Law holds for almost *any distribution*, minus some important cases we will encounter later.

### Example
____


Below is a diagram of the Law of Large numbers in action for three different sequences of Poisson random variables. 

 We sample `sample_size = 100000` Poisson random variables with parameter $\lambda = 4.5$. (Recall the expected value of a Poisson random variable is equal to its parameter.) We calculate the average for the first $n$ samples, for $n=1$ to `sample_size`. 

In [ ]:
sample_size = 100000
expected_value = lambda_val = 4.5
N_samples = np.arange(start = 1, stop = sample_size, step = 100)

plt.figure(figsize(12.5, 4))
for k in range(3):
    samples = Poisson(rate = lambda_val).sample(sample_shape = sample_size)
    partial_average = [np.mean(samples[:i]) for i in N_samples]        
    plt.plot(N_samples, partial_average, lw=1.5, label=f"Average of $n$ samples; seq. {k}")

plt.plot(N_samples, expected_value * np.ones_like(partial_average), 
         ls='--', label="True expected value", c='k')

plt.ylim(4.35, 4.65) 
plt.title("Convergence of the average of \n random variables to its expected value")
plt.ylabel("Average of $n$ samples")
plt.xlabel("Number of samples, $n$")
plt.legend();

Looking at the above plot, it is clear that when the sample size is small, there is greater variation in the average (compare how *jagged and jumpy* the average is initially, then *smooths* out). All three paths *approach* the value 4.5, but just flirt with it as $N$ gets large. Mathematicians and statistician have another name for *flirting*: convergence. 

Another very relevant question we can ask is *how quickly am I converging to the expected value?* Let's plot something new. For a specific $N$, let's do the above trials thousands of times and compute how far away we are from the true expected value, on average. But wait &mdash; *compute on average*? This is simply the law of large numbers again! For example, we are interested in, for a specific $N$, the quantity:

$$D(N) = \sqrt{E\left[ \left( \frac{1}{N}\sum_{i=1}^N Z_i  - 4.5 \right)^2 \right] }$$

The above formulae is interpretable as a distance away from the true value (on average), for some $N$. (We take the square root so the dimensions of the above quantity and our random variables are the same). As the above is an expected value, it can be approximated using the law of large numbers: instead of averaging $Z_i$, we calculate the following multiple times and average them:

$$ Y_k = \left( \frac{1}{N}\sum_{i=1}^NZ_i  - 4.5 \right)^2 $$

By computing the above many, $N_y$, times (remember, it is random), and averaging them:

$$ \frac{1}{N_Y} \sum_{k=1}^{N_Y} Y_k \rightarrow E[Y_k] = E\left[\left( \frac{1}{N}\sum_{i=1}^N Z_i - 4.5 \right)^2 \right]$$

Finally, taking the square root:

$$ \sqrt{\frac{1}{N_Y} \sum_{k=1}^{N_Y} Y_k} \approx D(N) $$ 

In [ ]:
N_Y = tf.constant(250)  # Use this many to approximate D(N)
N_array = np.arange(1000., 50000., 2500)  # Use this many samples in the approx. to the variance.
lambda_val = tf.constant(4.5) 
expected_value = tf.constant(4.5)  # For X ~ Poi(lambda) , E[X] = lambda


def D_N(n):
    """This function approx. D_n, the average variance of using n samples."""
    Z = Poisson(rate = lambda_val).sample(sample_shape = (int(n), int(N_Y)))
    mean_Z = np.mean(Z, axis = 0)
    return np.mean(np.sqrt(((mean_Z - expected_value) ** 2)))

D_N_results = tf.convert_to_tensor([D_N(n) for n in N_array])

plt.figure(figsize(12.5, 3))
plt.xlabel("$N$")
plt.ylabel("Expected squared-distance\nfrom true value")
plt.plot(N_array, D_N_results, lw=3, 
         label="Expected distance between\nexpected value and\naverage of $N$ random variables")
plt.plot(N_array, np.sqrt(expected_value) / np.sqrt(N_array), lw=2, ls='--', 
         label=r"$\frac{\sqrt{\lambda}}{\sqrt{N}}$" )
plt.legend()
plt.title("How 'fast' is the sample average converging?");

As expected, the expected distance between our sample average and the actual expected value shrinks as $N$ grows large. But also notice that the *rate* of convergence decreases, that is, we need only 10 000 additional samples to move from 0.020 to 0.015, a difference of 0.005, but *20 000* more samples to again decrease from 0.015  to 0.010, again only a 0.005 decrease.


It turns out we can measure this rate of convergence. Above I have plotted a second line, the function $\sqrt{\lambda}/\sqrt{N}$. This was not chosen arbitrarily. In most cases, given a sequence of random variable distributed like $Z$, the rate of convergence to $E[Z]$ of the Law of Large Numbers is 

$$ \frac{ \sqrt{ Var(Z) } }{\sqrt{N} }$$

This is useful to know: for a given large $N$, we know (on average) how far away we are from the estimate. On the other hand, in a Bayesian setting, this can seem like a useless result: Bayesian analysis is OK with uncertainty so what's the *statistical* point of adding extra precise digits? Though drawing samples can be so computationally cheap that having a *larger* $N$ is fine too. 

### How do we compute $Var(Z)$ though?

The variance is simply another expected value that can be approximated! Consider the following, once we have the expected value (by using the Law of Large Numbers to estimate it, denote it $\mu$), we can estimate the variance:

$$ \frac{1}{N}\sum_{i=1}^N (Z_i - \mu)^2 \rightarrow E[(Z - \mu)^2] = Var(Z) $$



### Expected values and probabilities 
There is an even less explicit relationship between expected value and estimating probabilities. Define the *indicator function*

$$\mathbb{1}_A(x) = 
\begin{cases} 1 &  x \in A \\\\
              0 &  else
\end{cases}
$$
Then, by the law of large numbers, if we have many samples $X_i$, we can estimate the probability of an event $A$, denoted $P(A)$, by:

$$ \frac{1}{N} \sum_{i=1}^N \mathbb{1}_A(X_i) \rightarrow E[\mathbb{1}_A(X)] =  P(A) $$

Again, this is fairly obvious after a moments thought: the indicator function is only 1 if the event occurs, so we are summing only the times the event occurs and dividing by the total number of trials  (consider how we usually approximate probabilities using frequencies). For example, suppose we wish to estimate the probability that a $Z \sim Exp(.5)$ is greater than 5, and we have many samples from a $Exp(.5)$ distribution. 


$$ P( Z > 5 ) =  \frac{1}{N}\sum_{i=1}^N \mathbb{1}_{z > 5 }(Z_i) $$

In [ ]:
N = 10000

print("Probability Estimate:", np.shape(np.where(Exponential(rate = .5).sample(sample_shape = (N,)) > 5))[1] / N)

### What does this all have to do with Bayesian statistics? 


*Point estimates*, to be introduced in the next chapter, in Bayesian inference are computed using expected values. In more analytical Bayesian inference, we would have been required to evaluate complicated expected values represented as multi-dimensional integrals. No longer. If we can sample from the posterior distribution directly, we simply need to evaluate averages. Much easier. If accuracy is a priority, plots like the ones above show how fast you are converging. And if further accuracy is  desired, just take more samples from the posterior. 

When is enough enough? When can you stop drawing samples from the posterior? That is the practitioners decision, and also dependent on the variance of the samples (recall from above a high variance means the average will converge slower). 

We also should understand when the Law of Large Numbers fails. As the name implies, and comparing the graphs above for small $N$, the Law is only true for large sample sizes. Without this, the asymptotic result is not reliable. Knowing in what situations the Law fails can give us *confidence in how unconfident we should be*. The next section deals with this issue.

### The Disorder of Small Numbers

The Law of Large Numbers is only valid as $N$ gets *infinitely* large: never truly attainable.  While the law is a powerful tool, it is foolhardy to apply it liberally. Our next example illustrates this.




### Example: Aggregated geographic data


Often data comes in aggregated form. For instance, data may be grouped by state, county, or city level. Of course, the population numbers vary per geographic area. If the data is an average of some characteristic of each the geographic areas, we must be conscious of the Law of Large Numbers and how it can *fail* for areas with small populations.

We will observe this on a toy dataset. Suppose there are five thousand counties in our dataset. Furthermore,  population number in each state are uniformly distributed between 100 and 1500. The way the population numbers are generated is irrelevant to the discussion, so we do not justify this. We are interested in measuring the average height of individuals per county. Unbeknownst to us, height does **not** vary across county, and each individual, regardless of the county he or she is currently living in, has the same distribution of what their height may be:

$$ \text{height} \sim \text{Normal}(\text{loc}=150, \text{scale}=15 ) $$

We aggregate the individuals at the county level, so we only have data for the *average in the county*. What might our dataset look like?

In [ ]:
plt.figure(figsize(12.5, 4))

std_height = 15.
mean_height = 150.
num_counties = 5000
smallest_population = 100
largest_population = 1500

population = np.random.randint(smallest_population, largest_population, num_counties)

# Our strategy to vectorise this problem will be to end-to-end concatenate the
# number of draws we need. Then we'll loop over the pieces.
d = Normal(loc = mean_height, scale =  1. / std_height)
x = d.sample(np.sum(population))
average_across_county_array = []
seen = 0
for p in population:
    average_across_county_array.append(np.mean(x[seen:seen+p]))
    seen += p
average_across_county = tf.stack(average_across_county_array)
# Locate the counties with the apparently most extreme average heights.
i_min, i_max = np.argmin(average_across_county), np.argmax(average_across_county)

# Plot population size vs. recorded average
plt.scatter(population, average_across_county, alpha=.5, c=TFColor[6])
plt.scatter([population[i_min], population[i_max]],
            tf.stack([average_across_county[i_min], average_across_county[i_max]]),
            s=60, marker='o', facecolors='none',
            edgecolors=TFColor[0], linewidths=1.5, 
            label='Extreme heights')

plt.xlim(smallest_population, largest_population)
plt.title("Average height vs. County Population")
plt.xlabel('County Population')
plt.ylabel('Average height in county')
plt.plot([smallest_population, largest_population], [mean_height, mean_height], color='k',
         label='True expected height', ls='--')
plt.legend(scatterpoints=1);

What do we observe? *Without accounting for population sizes* we run the risk of making an enormous inference error: if we ignored population size, we would say that the county with the shortest and tallest individuals have been correctly circled. But this inference is wrong for the following reason. These two counties do *not* necessarily have the most extreme heights. The error results from the calculated average of smaller populations not being a good reflection of the true expected value of the population (which in truth should be $\mu =150$). The sample size/population size/$N$, whatever you wish to call it,  is simply too small to invoke the Law of Large Numbers effectively. 

We provide more damning evidence against this inference. Recall the population numbers were uniformly distributed over 100 to 1500. Our intuition should tell us that the counties with the most extreme population heights should also be uniformly spread over 100 to 1500, and certainly independent of the county's population. Not so. Below are the population sizes of the counties with the most extreme heights.

In [ ]:
print("Population sizes of 10 'shortest' counties:", population[np.argsort(average_across_county)[:10]])
print("Population sizes of 10 'tallest' counties:", population[np.argsort(-average_across_county)[:10]])

Not at all uniform over 100 to 1500. This is an absolute failure of the Law of Large Numbers. 

### Example:  Kaggle's *U.S. Census Return Rate Challenge*

Below is data from the 2010 US census, which partitions populations beyond counties to the level of block groups (which are aggregates of city blocks or equivalents). The dataset is from a Kaggle machine learning competition some colleagues and I participated in. The objective was to predict the census letter mail-back rate of a group block, measured between 0 and 100, using census variables (median income, number of females in the block-group, number of trailer parks, average number of children etc.). Below we plot the census mail-back rate versus block group population:

In [ ]:
url = 'https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter4_TheGreatestTheoremNeverTold/data/census_data.csv'
filename = wget.download(url)
filename

In [ ]:
plt.figure(figsize(12.5, 6.5))
data = np.genfromtxt('census_data.csv',
                     skip_header = 1, 
                     delimiter = ',')
plt.scatter(data[:, 1], data[:, 0], alpha=.5, c=TFColor[6])
plt.title("Census mail-back rate vs population")
plt.ylabel('Mail-back rate')
plt.xlabel('Population of block-group')
plt.xlim(-100, 15e3)
plt.ylim(-5, 105)

i_min = np.argmin(data[:, 0])
i_max = np.argmax(data[:, 0])
 
plt.scatter([data[i_min, 1], data[i_max, 1]], 
            [data[i_min, 0], data[i_max, 0]],
            s=60, marker='o', facecolors='none',
            edgecolors=TFColor[0], linewidths=1.5, 
            label="Most extreme points")

plt.legend(scatterpoints=1);

The above is a classic phenomenon in statistics. I say *classic* referring to the "shape" of the scatter plot above. It follows a classic triangular form, that tightens as we increase the sample size (as the Law of Large Numbers becomes more exact). 

I am perhaps overstressing the point and maybe I should have titled the book *"You don't have big data problems!"*, but here again is an example of the trouble with *small datasets*, not big ones. Simply, small datasets cannot be processed using the Law of Large Numbers. Compare with applying the Law without hassle to big datasets (ex. big data). I mentioned earlier that paradoxically big data prediction problems are solved by relatively simple algorithms. The paradox is partially resolved by understanding that the Law of Large Numbers creates solutions that are *stable*, i.e. adding or subtracting a few data points will not affect the solution much. On the other hand, adding or removing data points to a small dataset can create very different results. 

For further reading on the hidden dangers of the Law of Large Numbers, I would highly recommend the excellent manuscript [The Most Dangerous Equation](http://nsm.uh.edu/~dgraur/niv/TheMostDangerousEquation.pdf). 

### Example: How to order Reddit submissions

You may have disagreed with the original statement that the Law of Large numbers is known to everyone, but only implicitly in our subconscious decision making. Consider ratings on online products: how often do you trust an average 5-star rating if there is only 1 reviewer? 2 reviewers? 3 reviewers? We implicitly understand that with such few reviewers that the average rating is **not** a good reflection of the true value of the product.

This has created flaws in how we sort items, and more generally, how we compare items. Many people have realized that sorting online search results by their rating, whether the objects be books, videos, or online comments, return poor results. Often the seemingly top videos or comments have perfect ratings only from a few enthusiastic fans, and truly more quality videos or comments are hidden in later pages with *falsely-substandard* ratings of around 4.8. How can we correct this?

Consider the popular site Reddit (I purposefully did not link to the website as you would never come back). The site hosts links to stories or images, called submissions, for people to comment on. Redditors can vote up or down on each submission (called upvotes and downvotes). Reddit, by default, will sort submissions to a given subreddit by Hot, that is, the submissions that have the most upvotes recently.

<img src="http://i.imgur.com/3v6bz9f.png" />


How would you determine which submissions are the best? There are a number of ways to achieve this:

1. *Popularity*: A submission is considered good if it has many upvotes. A problem with this model is that a submission with hundreds of upvotes, but thousands of downvotes. While being very *popular*, the submission is likely more controversial than best.
2. *Difference*: Using the *difference* of upvotes and downvotes. This solves the above problem, but fails when we consider the temporal nature of submission. Depending on when a submission is posted, the website may be experiencing high or low traffic. The difference method will bias the *Top* submissions to be the those made during high traffic periods, which have accumulated more upvotes than submissions that were not so graced, but are not necessarily the best.
3. *Time adjusted*:  Consider using Difference divided by the age of the submission. This creates a *rate*, something like *difference per second*, or *per minute*. An immediate counter-example is, if we use per second, a 1 second old submission with 1 upvote would be better than a 100 second old submission with 99 upvotes. One can avoid this by only considering at least t second old submission. But what is a good t value? Does this mean no submission younger than t is good? We end up comparing unstable quantities with stable quantities (young vs. old submissions).
3. *Ratio*: Rank submissions by the ratio of upvotes to total number of votes (upvotes plus downvotes). This solves the temporal issue, such that new submissions who score well can be considered Top just as likely as older submissions, provided they have many upvotes to total votes. The problem here is that a submission with a single upvote (ratio = 1.0) will beat a submission with 999 upvotes and 1 downvote (ratio = 0.999), but clearly the latter submission is *more likely* to be better.

I used the phrase *more likely* for good reason. It is possible that the former submission, with a single upvote, is in fact a better submission than the later with 999 upvotes. The hesitation to agree with this is because we have not seen the other 999 potential votes the former submission might get. Perhaps it will achieve an additional 999 upvotes and 0 downvotes and be considered better than the latter, though not likely.

What we really want is an estimate of the *true upvote ratio*. Note that the true upvote ratio is not the same as the observed upvote ratio: the true upvote ratio is hidden, and we only observe upvotes vs. downvotes (one can think of the true upvote ratio as "what is the underlying probability someone gives this submission a upvote, versus a downvote"). So the 999 upvote/1 downvote submission probably has a true upvote ratio close to 1, which we can assert with confidence thanks to the Law of Large Numbers, but on the other hand we are much less certain about the true upvote ratio of the submission with only a single upvote. Sounds like a Bayesian problem to me.



One way to determine a prior on the upvote ratio is to look at the historical distribution of upvote ratios. This can be accomplished by scraping Reddit's submissions and determining a distribution. There are a few problems with this technique though:

1. Skewed data:  The vast majority of submissions have very few votes, hence there will be many submissions with ratios near the extremes (see the "triangular plot" in the above Kaggle dataset), effectively skewing our distribution to the extremes. One could try to only use submissions with votes greater than some threshold. Again, problems are encountered. There is a tradeoff between number of submissions available to use and a higher threshold with associated ratio precision. 
2. Biased data: Reddit is composed of different subpages, called subreddits. Two examples are *r/aww*, which posts pics of cute animals, and *r/politics*. It is very likely that the user behaviour towards submissions of these two subreddits are very different: visitors are likely friendly and affectionate in the former, and would therefore upvote submissions more, compared to the latter, where submissions are likely to be controversial and disagreed upon. Therefore not all submissions are the same. 


In light of these, I think it is better to use a `Uniform` prior.


With our prior in place, we can find the posterior of the true upvote ratio. The Python script below will scrape the best posts from the `showerthoughts` community on Reddit. This is a text-only community so the title of each post *is* the post.

#### Setting up the `Praw` Reddit API

Use of the `praw` package for retrieving data from Reddit does require some private information on your Reddit account. As such, we are not releasing the secret keys and reddit account passwords that we originally used for the code cell below. Fortunately, we've provided detailed information on how to set up the next code cell with your custom information.

#### Register your Application on Reddit

1. Log into your Reddit account.

2. Click the down arrow to the right of your name, then click the Preferences button.

<img src="https://cdn-images-1.medium.com/max/1600/1*YMLEuY0AXaSVW2YJAbUiaQ.png" width="250">

3. Click the app tab.

<img src="https://cdn-images-1.medium.com/max/1600/1*TDDvEVTSWTERxUw2ZrJDWA.png" width="600">

4. Click the create another app button at the bottom left of your screen.

5. Populate your script with the required fields. Refer to the screen shot below:

<img src="https://cdn-images-1.medium.com/max/1600/1*duC42-xMothcka2WXLKGiw.png" width="600">

6. Hit the create app button once you have populated all fields. You should now have a script which resembles the following:

<img src="https://cdn-images-1.medium.com/max/1600/1*v_et9Ei38h0zZ0SdMCV0bQ.png" width="600">


NOTE: Certain components of the `reddit = praw.Reddit("BasyesianMethodsForHackers")` code have been intentionally omitted. This is because praw requires a user ID for accessing Reddit. the praw function follows the following format:
```python
reddit = praw.Reddit(client_id = 'PERSONAL_USE_SCRIPT_14_CHARS',
                     client_secret = 'SECRET_KEY_27_CHARS ',
                     user_agent = 'YOUR_APP_NAME',
                     username = 'YOUR_REDDIT_USER_NAME',
                     password = 'YOUR_REDDIT_LOGIN_PASSWORD')
```
For help with creating a Reddit instance, visit
https://praw.readthedocs.io/en/latest/code_overview/reddit_instance.html.

For help on configuring PRAW, visit
https://praw.readthedocs.io/en/latest/getting_started/configuration.html.

In [ ]:
# Reddit API setup
from IPython.core.display import Image
import praw


enter_client_id = 'ZhGqHeR1zTM9fg'
enter_client_secret = 'keZdvIa1Ge257NKEm3v-eGEdv8M'
enter_user_agent = 'bayesian_app'
enter_username = 'ThisIsJustADemo'
enter_password = 'EnterYourOwnInfoHere'

# e.g. "showerthoughts", "todayilearned", "worldnews", "science", "lifeprotips", "nottheonion"
subreddit_name = 'showerthoughts'

reddit = praw.Reddit(client_id = enter_client_id,
                     client_secret = enter_client_secret,
                     user_agent = enter_user_agent,
                     username = enter_username,
                     password = enter_password)
subreddit  = reddit.subreddit(subreddit_name)

# Go by timespan - 'hour', 'day', 'week', 'month', 'year', 'all'
# might need to go longer than an hour to get entries...

# e.g. 'hour', 'day', 'week', 'month', 'year', 'all'
timespan = 'day'

top_submissions = subreddit.top(timespan)

# Adding a number to the inside of int() call will get the ith top post.
ith_top_post = 2
n_sub = int(ith_top_post)

i = 0
while i < n_sub:
    top_submission = next(top_submissions)
    i += 1

top_post = top_submission.title

upvotes = []
downvotes = []
contents = []

for sub in top_submissions:
    try:
        ratio = sub.upvote_ratio
        ups = int(round((ratio * sub.score) / (2 * ratio - 1))
                  if ratio != .5 else round(sub.score / 2))
        upvotes.append(ups)
        downvotes.append(ups - sub.score)
        contents.append(sub.title)
    except Exception as e:
        continue

votes = np.array([upvotes, downvotes]).T

print("Post contents: \n")
print(top_post)

Above is the top post as well as some other sample posts:

In [ ]:
# Contents: an array of the text from the last 100 top submissions to a subreddit
# votes: a 2d numpy array of upvotes, downvotes for each submission.
n_submissions = len(votes)
submissions = Uniform(low = 0., high = float(n_submissions)).sample(sample_shape = (4,))
submissions = submissions.astype(np.i32)

print(f"Some Submissions (out of {n_submissions} total)\n-----------")
for i in submissions:
    print('"' + contents[i] + '"')
    print("upvotes/downvotes:", votes[i, :], "\n")

For a given true upvote ratio $p$ and $N$ votes, the number of upvotes will look like a Binomial random variable with parameters $p$ and $N$. (This is because of the equivalence between upvote ratio and probability of upvoting versus downvoting, out of $N$ possible votes/trials). We create a function that performs Bayesian inference on $p$, for a particular submission's upvote/downvote pair.

In [ ]:
def joint_log_prob(upvotes, N, test_upvote_ratio):
    """
    Args:
      upvotes: observed upvotes for a submission
      N : observed upvotes+downvotes for the submission
      test_upvote_ratio: hypothesized value for true value of upvote ratio
    Returns: 
      Joint log probability optimization function to compute true upvote ratio.
    """
    # Use a uniform prior
    rv_upvote_ratio = Uniform(low = 0., high = 1.)
    rv_observations = Binomial(total_count = float(N),
                               probs = test_upvote_ratio)
    return (
          rv_upvote_ratio.log_prob(test_upvote_ratio)
        + rv_observations.log_prob(float(upvotes)))

in some cases we might want to run someting like an HMC for multiple, or a variable number, of inputs. Loops are common examples of this. Here we define our function for setting up an HMC that can take in different numbers of upvotes and/or downvotes.

In [ ]:
def posterior_upvote_ratio(upvotes, downvotes):
    num_results = 5000
    num_burnin_steps = 1000
    N = float(upvotes) + float(downvotes)

    # Initialise the step_size (it will be automatically adapted.)
    step_size = tf.Variable(
        initial_value = tf.constant(.5, dtype = f32),
        trainable = False,
        name = 'step_size') 

    # Set the chain's start state.
    initial_chain_state = [
        .5 * np.ones((), dtype = f32)]

    # Since HMC operates over unconstrained space, we need to transform the
    # samples so they live in real-space.
    unconstraining_bijectors = [
        tfb.Sigmoid()          ]

    # Define a closure over our joint_log_prob.
    unnormalised_posterior_log_prob = lambda *args: joint_log_prob(upvotes, N, *args)

    kernel = tfp.mcmc.TransformedTransitionKernel(
        inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = unnormalised_posterior_log_prob,
            num_leapfrog_steps = 2,
            step_size = step_size,
            state_gradients_are_stopped = True),
        bijector = unconstraining_bijectors)

    kernel = tfp.mcmc.SimpleStepSizeAdaptation(
      inner_kernel = kernel,
      num_adaptation_steps = int(num_burnin_steps * .8))

    # Sample from the chain
    # TODO Rename 'posterior_upvote_ratio' since it's duplicate to the fn's name.
    (posterior_upvote_ratio,), kernel_results = tfp.mcmc.sample_chain(
        num_results = num_results,
        num_burnin_steps = num_burnin_steps,
        current_state = initial_chain_state,
        kernel = kernel)
    
    return posterior_upvote_ratio, kernel_results

In [ ]:
plt.figure(figsize(11., 8))
posteriors = []
colours = ['#5DA5DA', '#F15854', '#B276B2', '#60BD68', '#F17CB0']

for i in range(len(submissions)):
    j = submissions[i]
    posteriors.append(posterior_upvote_ratio(votes[j, 0], votes[j, 1])[0])
    plt.hist(posteriors[i], bins=10, alpha=.9, 
             histtype='step', color=colours[i], lw=3,
             label=f'({} up:{} down)\n{}...' % (votes[j, 0], votes[j, 1], contents[j][:50]))
    plt.hist(posteriors[i], bins=10, alpha=.2, 
             histtype='stepfilled', color=colours[i], lw=3)
    
plt.legend(loc='upper left')
plt.xlim(0, 1)
plt.title("Posterior distributions of upvote ratios on different submissions");

Some distributions are very tight, others have very long tails (relatively speaking), expressing our uncertainty with what the true upvote ratio might be.

#### Sorting!

We have been ignoring the goal of this exercise: how do we sort the submissions from *best to worst*? Of course, we cannot sort distributions, we must sort scalar numbers. There are many ways to distill a distribution down to a scalar: expressing the distribution through its expected value, or mean, is one way. Choosing the mean is a bad choice though. This is because the mean does not take into account the uncertainty of distributions.

I  suggest using the *95% least plausible value*, defined as the value such that there is only a 5% chance the true parameter is lower (think of the lower bound on the 95% credible region). Below are the posterior distributions with the 95% least-plausible value plotted:

In [ ]:
N = posteriors[0].shape[0]
lower_limits = []
for i in range(len(submissions)):
    j = submissions[i]
    plt.hist(posteriors[i], bins=20, alpha=.9, 
             histtype='step', color=colours[i], lw=3,
             label=f'({} up:{} down)\n{}...' % (votes[j, 0], votes[j, 1], contents[j][:50]))
    plt.hist(posteriors[i], bins=20, alpha=.2, 
             histtype='stepfilled', color=colours[i], lw=3)
    v = np.sort(posteriors[i] )[ int(.05 * N)]
    plt.vlines(v, 0, 30, color=colours[i], linestyles='--', lw=3)
    lower_limits.append(v)
    plt.legend(loc='upper left')

plt.legend(loc='upper left')
plt.title("Posterior distributions of upvote ratios on different submissions");
order = np.argsort(-np.array(lower_limits))
print(order, lower_limits)

The best submissions, according to our procedure, are the submissions that are *most-likely* to score a high percentage of upvotes. Visually those are the submissions with the 95% least plausible value close to 1.

Why is sorting based on this quantity a good idea? By ordering by the 95% least plausible value, we are being the most conservative with what we think is best.  When using the lower-bound of the 95% credible interval, we believe with high certainty that the 'true upvote ratio' is at the very least equal to this value (or greater), thereby ensuring that the best submissions are still on top. Under this ordering, we impose the following very natural properties:

1. given two submissions with the same observed upvote ratio, we will assign the submission with more votes as better (since we are more confident it has a higher ratio).
2. given two submissions with the same number of votes, we still assign the submission with more upvotes as *better*.

#### But this is too slow for real-time!

I agree, computing the posterior of every submission takes a long time, and by the time you have computed it, likely the data has changed. I delay the mathematics to the appendix, but I suggest using the following formula to compute the lower bound very fast.

$$ \frac{a}{a + b} - 1.65\sqrt{ \frac{ab}{ (a+b)^2(a + b +1 ) } }$$

where 
$$
\begin{align}
& a = 1 + u \\
& b = 1 + d \\
\end{align}
$$
$u$ is the number of upvotes, and $d$ is the number of downvotes. The formula is a shortcut in Bayesian inference, which will be further explained in Chapter 6 when we discuss priors in more detail.

In [ ]:
def intervals(u, d):
    a = tf.add(1., u)
    b = tf.add(1., d)
    mu = tf.divide(x = a, y = tf.add(1., u))
    std_err = 1.65 * np.sqrt((a * b) / ((a + b) ** 2 * (a + b + 1.)))
    
    return mu, std_err
  
print("Approximate lower bounds:")
posterior_mean, std_err  = intervals(votes[:, 0], votes[:, 1])
lb = posterior_mean - std_err
print(lb)
print("\n")
print("Top 40 Sorted according to approximate lower bounds:")
print("\n")
order = tf.nn.top_k(lb, k=lb.shape[0], sorted=True)
ordered_contents = []
for i, N in enumerate(order.values[:40]):
    ordered_contents.append(contents[i])
    print(votes[i, 0], votes[i, 1], contents[i])
    print("-------------")

We can view the ordering visually by plotting the posterior mean and bounds, and sorting by the lower bound. In the plot below, notice that the left error-bar is sorted (as we suggested this is the best way to determine an ordering), so the means, indicated by dots, do not follow any strong pattern. 

In [ ]:
r_order = order.indices[::-1][-40:]
ratio_range = np.range(len(r_order) - 1, -1, -1) 
r_order_vals = order.values[::-1][-40:]
plt.errorbar(r_order_vals, 
             np.arange(len(r_order)), 
             xerr=tf.gather(std_err, r_order), capsize=0, fmt='o',
             color=TFColor[0])
plt.xlim(.3, 1)
plt.yticks(ratio_range, map(lambda x: x[:30].replace('\n', ''), ordered_contents));

In the graphic above, you can see why sorting by mean would be sub-optimal.

### Extension to Starred rating systems

The above procedure works well for upvote-downvotes schemes, but what about systems that use star ratings, e.g. 5 star rating systems. Similar problems apply with simply taking the average: an item with two perfect ratings would beat an item with thousands of perfect ratings, but a single sub-perfect rating. 


We can consider the upvote-downvote problem above as binary: 0 is a downvote, 1 if an upvote. A $N$-star rating system can be seen as a more continuous version of above, and we can set $n$ stars rewarded is equivalent to rewarding $\frac{n}{N}$. For example, in a 5-star system, a 2 star rating corresponds to 0.4. A perfect rating is a 1. We can use the same formula as before, but with $a,b$ defined differently:


$$ \frac{a}{a + b} - 1.65\sqrt{ \frac{ab}{ (a+b)^2(a + b +1 ) } }$$

where 
$$
\begin{align}
& a = 1 + S \\
& b = 1 + N - S \\
\end{align}
$$
where $N$ is the number of users who rated, and $S$ is the sum of all the ratings, under the equivalence scheme mentioned above. 

### Example: Counting Github stars

What is the average number of stars a Github repository has? How would you calculate this? There are over 6 million respositories, so there is more than enough data to invoke the Law of Large numbers. Let's start pulling some data.

In [ ]:
url = 'https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter3_MCMC/data/github_data.csv'
filename = wget.download(url)
filename

In [ ]:
# Github data scrapper
# See documentation_url: https://developer.github.com/v3/
from requests import get

"""
variables of interest:
    indp. variables
    - language, given as a binary variable. Need 4 positions for 5 langagues
    - #number of days created ago, 1 position
    - has wiki? Boolean, 1 position
    - followers, 1 position
    - following, 1 position
    - constant
    
    dep. variables
    -stars/watchers
    -forks
"""


MAX = 8000000
today =  datetime.datetime.today()
randint = np.random.randint
N = 20  # Sample size
auth = ('mikeshwe', 'kick#Ass1')

language_mappings = {'Python': 0, 'JavaScript': 1, 'Ruby': 2, 'Java': 3, 'Shell': 4, 'PHP': 5}

# Define data matrix: 
X = np.zeros((N, 12), dtype = int)

for i in range(N):
    is_fork = True
    is_valid_language = False
    
    while is_fork or not is_valid_language:
        is_fork = True
        is_valid_language = False
        
        params = {'since': randint(0, MAX)}
        r = get('https://api.github.com/repositories', params=params, auth=auth)
        results = json.loads(r.text)[0]
        # im only interested in the first one, and if it is not a fork.
        # print(results)
        is_fork = results['fork']
        
        r = get(results['url'], auth=auth)
        
        # Check the language
        repo_results = json.loads(r.text)
        try: 
            language_mappings[repo_results['language']]
            is_valid_language = True
        except:
            pass

    # languages 
    X[i, language_mappings[repo_results['language']]] = 1
    
    # delta time
    X[i, 6] = (today - datetime.datetime.strptime(repo_results['created_at'][:10], '%Y-%m-%d')).days
    
    # haswiki
    X[i, 7] = repo_results['has_wiki']
    
    # get user information
    r = get(results['owner']['url'], auth=auth)
    user_results = json.loads(r.text)
    X[i, 8] = user_results['following']
    X[i, 9] = user_results['followers']
    
    # get dep. data
    X[i, 10] = repo_results['watchers_count']
    X[i, 11] = repo_results['forks_count']
    print(" -------------- ")
    print(i, ':', results['full_name'], repo_results['language'], repo_results['watchers_count'], repo_results['forks_count']) 
    print(" -------------- ") 
    
np.savetxt('github_data.csv', X, delimiter=',', fmt='%d')

### Conclusion

While the Law of Large Numbers is cool, it is only true so much as its name implies: with large sample sizes only. We have seen how our inference can be affected by not considering *how the data is shaped*. 

1. By (cheaply) drawing many samples from the posterior distributions, we can ensure that the Law of Large Number applies as we approximate expected values (which we will do in the next chapter).

2. Bayesian inference understands that with small sample sizes, we can observe wild randomness. Our posterior distribution will reflect this by being more spread rather than tightly concentrated. Thus, our inference should be correctable.

3. There are major implications of not considering the sample size, and trying to sort objects that are unstable leads to pathological orderings. The method provided above solves this problem.


### Appendix

##### Derivation of sorting submissions formula

Basically what we are doing is using a Beta prior (with parameters $a=1, b=1$, which is a uniform distribution), and using a Binomial likelihood with observations $u, N = u+d$. This means our posterior is a Beta distribution with parameters $a' = 1 + u, b' = 1 + (N - u) = 1+d$. We then need to find the value, $x$, such that 0.05 probability is less than $x$. This is usually done by inverting the CDF ([Cumulative Distribution Function](http://en.wikipedia.org/wiki/Cumulative_Distribution_Function)), but the CDF of the beta, for integer parameters, is known but is a large sum [3]. 

We instead use a Normal approximation. The mean of the Beta is $\mu = a'/(a'+b')$ and the variance is 

$$\sigma^2 = \frac{a'b'}{ (a' + b')^2(a'+b'+1) }$$

Hence we solve the following equation for $x$ and have an approximate lower bound. 

$$ 0.05 = \Phi\left( \frac{(x - \mu)}{\sigma}\right) $$ 

$\Phi$ being the [cumulative distribution for the normal distribution](http://en.wikipedia.org/wiki/Normal_distribution#Cumulative_distribution)

##### Exercises

1\. How would you estimate the quantity $E\left[ \cos{X} \right]$, where $X \sim \text{Exp}(4)$? What about $E\left[ \cos{X} | X \lt 1\right]$, i.e. the expected value *given* we know $X$ is less than 1? Would you need more samples than the original samples size to be equally accurate?

In [ ]:
## Enter code here
%%time
exp = Exponential(rate = 4.)
N = 10000
X = exp.sample(sample_shape = int(N))
print(X)
  
## ...

2\. The following table was located in the paper "Going for Three: Predicting the Likelihood of Field Goal Success with Logistic Regression" [2]. The table ranks football field-goal kickers by their percent of non-misses. What mistake have the researchers made?

-----

####  Kicker Careers Ranked by Make Percentage
<table><tbody><tr><th>Rank </th><th>Kicker </th><th>Make % </th><th>Number  of Kicks</th></tr><tr><td>1 </td><td>Garrett Hartley </td><td>87.7 </td><td>57</td></tr><tr><td>2</td><td> Matt Stover </td><td>86.8 </td><td>335</td></tr><tr><td>3 </td><td>Robbie Gould </td><td>86.2 </td><td>224</td></tr><tr><td>4 </td><td>Rob Bironas </td><td>86.1 </td><td>223</td></tr><tr><td>5</td><td> Shayne Graham </td><td>85.4 </td><td>254</td></tr><tr><td>… </td><td>… </td><td>…</td><td> </td></tr><tr><td>51</td><td> Dave Rayner </td><td>72.2 </td><td>90</td></tr><tr><td>52</td><td> Nick Novak </td><td>71.9 </td><td>64</td></tr><tr><td>53 </td><td>Tim Seder </td><td>71.0 </td><td>62</td></tr><tr><td>54 </td><td>Jose Cortez </td><td>70.7</td><td> 75</td></tr><tr><td>55 </td><td>Wade Richey </td><td>66.1</td><td> 56</td></tr></tbody></table>

In August 2013, [a popular post](http://bpodgursky.wordpress.com/2013/08/21/average-income-per-programming-language/) on the average income per programmer of different languages was trending. Here's the summary chart: (reproduced without permission, cause when you lie with stats, you gunna get the hammer). What do you notice about the extremes?

------

#### Average household income by programming language

<table >
 <tr><td>Language</td><td>Average Household Income ($)</td><td>Data Points</td></tr>
 <tr><td>Puppet</td><td>87,589.29</td><td>112</td></tr>
 <tr><td>Haskell</td><td>89,973.82</td><td>191</td></tr>
 <tr><td>PHP</td><td>94,031.19</td><td>978</td></tr>
 <tr><td>CoffeeScript</td><td>94,890.80</td><td>435</td></tr>
 <tr><td>VimL</td><td>94,967.11</td><td>532</td></tr>
 <tr><td>Shell</td><td>96,930.54</td><td>979</td></tr>
 <tr><td>Lua</td><td>96,930.69</td><td>101</td></tr>
 <tr><td>Erlang</td><td>97,306.55</td><td>168</td></tr>
 <tr><td>Clojure</td><td>97,500.00</td><td>269</td></tr>
 <tr><td>Python</td><td>97,578.87</td><td>2314</td></tr>
 <tr><td>JavaScript</td><td>97,598.75</td><td>3443</td></tr>
 <tr><td>Emacs Lisp</td><td>97,774.65</td><td>355</td></tr>
 <tr><td>C#</td><td>97,823.31</td><td>665</td></tr>
 <tr><td>Ruby</td><td>98,238.74</td><td>3242</td></tr>
 <tr><td>C++</td><td>99,147.93</td><td>845</td></tr>
 <tr><td>CSS</td><td>99,881.40</td><td>527</td></tr>
 <tr><td>Perl</td><td>100,295.45</td><td>990</td></tr>
 <tr><td>C</td><td>100,766.51</td><td>2120</td></tr>
 <tr><td>Go</td><td>101,158.01</td><td>231</td></tr>
 <tr><td>Scala</td><td>101,460.91</td><td>243</td></tr>
 <tr><td>ColdFusion</td><td>101,536.70</td><td>109</td></tr>
 <tr><td>Objective-C</td><td>101,801.60</td><td>562</td></tr>
 <tr><td>Groovy</td><td>102,650.86</td><td>116</td></tr>
 <tr><td>Java</td><td>103,179.39</td><td>1402</td></tr>
 <tr><td>XSLT</td><td>106,199.19</td><td>123</td></tr>
 <tr><td>ActionScript</td><td>108,119.47</td><td>113</td></tr>
</table>

### References

1. Wainer, Howard. *The Most Dangerous Equation*. American Scientist, Volume 95.
2. Clarck, Torin K., Aaron W. Johnson, and Alexander J. Stimpson. "Going for Three: Predicting the Likelihood of Field Goal Success with Logistic Regression." (2013): n. page. [Web](http://www.sloansportsconference.com/wp-content/uploads/2013/Going%20for%20Three%20Predicting%20the%20Likelihood%20of%20Field%20Goal%20Success%20with%20Logistic%20Regression.pdf). 20 Feb. 2013.
3. http://en.wikipedia.org/wiki/Beta_function#Incomplete_beta_function